In [1]:
# mac475의 ipython 표준 style을 적용함
from IPython.core.display import HTML
styles = open("../styles/custom.css", "r").read()
HTML( styles )

#1. tube-bill_merged와 spces의 merge

- spec dataset의 merge 필요성

    → tube_bill_merged dataset만 활용시 modeling 활용 feature 제약, tube_assembly_id join하여 specs dataset내의 tube 속성정보 modeling에 활용
    
    → spec1~10, 10개 속성 : 상세한 의미는 알 수 없음
    
- 단, specs dataset을 보완필요 있는지 사전확인후, 적절한 작업의 병행필요
	
<img src="images/02.tube-bill-specs.png" style="display:inline; width: 60%" />

#2. specs 주요 feature 검사

In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv( './dataset/01.original.dataset/specs.csv' )

##2.1 specs 누락 feature 확인

In [3]:
df.count()    # spec의 전체적인 data 충실도를 확인

tube_assembly_id    21198
spec1                7129
spec2                6844
spec3                5840
spec4                4154
spec5                2921
spec6                2071
spec7                 535
spec8                 106
spec9                  20
spec10                  1
dtype: int64

* 확인결과
    
    - N/A

.

##2.2 각 features의 unique 구성 확인

In [4]:
def get_unique_elements_sorted( name, sz ) :    # series내의 unique 정보를 sorted 제공
    tmp = sz.sort( axis = 0, ascending = True, inplace = False ).unique()
#     print( 'name : ', name, tmp, 'count : ', len( tmp ) )
    sz = None
    tmp = None    

In [5]:
list_cols = df.columns.values.tolist()    # columns

for col in list_cols[1:] :    # 일반적으로 0번째 col은 pk : id이므로 uniqueness 조사 필요성 없음
    get_unique_elements_sorted( col, df[ col ].copy() )

<font color = 'red'>* unique 확인결과, 이상 data 문제없음</font>

#3. Derived feature 추출작업

In [6]:
def calculate_spec_type_count( p_df ) :    # spec type의 count를 계산
    spec_count = 0    # spec count 초기화
    for i in range( 1, 11 ) :
        spec_str = 'spec' + str( i )
        
        if pd.notnull( p_df[ spec_str ] )  :
            spec_count += 1
    return spec_count

In [7]:
for i in range( 1, 11 ) :    # spec feature들에 대한 type casting 수행
    spec_str = 'spec' + str( i )
    df[ spec_str ].astype( str )

In [8]:
df[ 'spec_type_count' ] = df.apply( calculate_spec_type_count, axis = 1 )    # spec type의 개수 확보

In [9]:
df.fillna( 'NONE', inplace = True )

In [10]:
df.head( 3 )

,tube_assembly_id,spec1,spec2,spec3,spec4,spec5,spec6,spec7,spec8,spec9,spec10,spec_type_count
0,TA-00001,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,0
1,TA-00002,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,0
2,TA-00003,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,NONE,0


In [11]:
def get_spec_feature( p_df, p_str ) :
    ret = 0
    for i in range( 1, 11 ) :
        data = p_df[ 'spec' + str( i ) ]
        if data != 'NONE' :
            if data == p_str :
                ret = 1
                break
            else :
                if data > p_str :
                    break
        else :
            break
    return ret

In [12]:
for i in range( 1, 97 ) :    # 일단, SP-0001 ~ SP-0096을 0으로 초기화한다
    sp_str = 'SP-00'
    
    if i < 10 :
        sp_str += '0'
    sp_str += str( i )
    df[ sp_str ] = df.apply( get_spec_feature, axis = 1, args = (sp_str,) )

In [13]:
df.head( 3 )[10:]

,tube_assembly_id,spec1,spec2,spec3,spec4,spec5,spec6,spec7,spec8,spec9,...,SP-0087,SP-0088,SP-0089,SP-0090,SP-0091,SP-0092,SP-0093,SP-0094,SP-0095,SP-0096


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21198 entries, 0 to 21197
Columns: 108 entries, tube_assembly_id to SP-0096
dtypes: int64(97), object(11)
memory usage: 17.6+ MB


In [15]:
df.count()

tube_assembly_id    21198
spec1               21198
spec2               21198
spec3               21198
spec4               21198
spec5               21198
spec6               21198
spec7               21198
spec8               21198
spec9               21198
spec10              21198
spec_type_count     21198
SP-0001             21198
SP-0002             21198
SP-0003             21198
...
SP-0082    21198
SP-0083    21198
SP-0084    21198
SP-0085    21198
SP-0086    21198
SP-0087    21198
SP-0088    21198
SP-0089    21198
SP-0090    21198
SP-0091    21198
SP-0092    21198
SP-0093    21198
SP-0094    21198
SP-0095    21198
SP-0096    21198
Length: 108, dtype: int64

#4. 정련된 specs 저장

In [16]:
df.to_csv( './dataset/02.ml.verified.dataset/specs.verified.csv', index = False )
del( df )

.

#5. Merge 시도

In [17]:
# df_tube_bill = pd.read_csv( './dataset/03.merged/tube_bill_merged.csv', dtype = { 'component_id_8' : str } )
df_tube_bill = pd.read_csv( './dataset/03.merged/tube_bill_merged.csv' )
df_specs = pd.read_csv( './dataset/02.ml.verified.dataset/specs.verified.csv' )

In [18]:
len( df_tube_bill ), len( df_specs )

(21198, 21198)

→ inner join 시도

In [19]:
df_tube_bill_specs = df_tube_bill.merge( df_specs, how = 'inner', on = 'tube_assembly_id' )

In [20]:
df_tube_bill_specs.head( 10 )

,tube_assembly_id,material_id,diameter,wall,length,num_bends,bend_radius,end_a_1x,end_a_2x,end_x_1x,...,SP-0087,SP-0088,SP-0089,SP-0090,SP-0091,SP-0092,SP-0093,SP-0094,SP-0095,SP-0096
0,TA-00001,SP-0035,12.70,1.65,164,5,38.10,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,TA-00002,SP-0019,6.35,0.71,137,8,19.05,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,TA-00003,SP-0019,6.35,0.71,127,7,19.05,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,TA-00004,SP-0019,6.35,0.71,137,9,19.05,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,TA-00005,SP-0029,19.05,1.24,109,4,50.80,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,TA-00006,SP-0029,19.05,1.24,79,4,50.80,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,TA-00007,SP-0035,12.70,1.65,202,5,38.10,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,TA-00008,SP-0039,6.35,0.71,174,6,19.05,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,TA-00009,SP-0029,25.40,1.65,135,4,63.50,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,TA-00010,SP-0046,42.70,4.80,290,4,110.00,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [21]:
df_tube_bill_specs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21198 entries, 0 to 21197
Columns: 178 entries, tube_assembly_id to SP-0096
dtypes: float64(50), int64(106), object(22)
memory usage: 28.9+ MB


In [22]:
df_tube_bill_specs.count()

tube_assembly_id    21198
material_id         21198
diameter            21198
wall                21198
length              21198
num_bends           21198
bend_radius         21198
end_a_1x            21198
end_a_2x            21198
end_x_1x            21198
end_x_2x            21198
end_a               21198
end_x               21198
num_boss            21198
num_bracket         21198
...
SP-0082    21198
SP-0083    21198
SP-0084    21198
SP-0085    21198
SP-0086    21198
SP-0087    21198
SP-0088    21198
SP-0089    21198
SP-0090    21198
SP-0091    21198
SP-0092    21198
SP-0093    21198
SP-0094    21198
SP-0095    21198
SP-0096    21198
Length: 178, dtype: int64

In [23]:
df_tube_bill_specs.to_csv( './dataset/03.merged/tube_bill_specs_merged.csv', index = False )

.

#6. spec_meta 생성
    
    * specs의 경우, meta가 없어, label encoding시 진행불가능하므로, specs내의 각 data를 활용하여, spec_meta를 생성
    * 
    

In [24]:
df = pd.read_csv( './dataset/01.original.dataset/specs.csv' )

In [25]:
list_cols = df.columns.values.tolist()    # columns
list_spec = [ 'NONE' ]

for col in list_cols[1:] :    # specs의 column별로 순환하며, 내부 data를 sorting/ unique확보하여 배열담고, 이를 담은 list 확보
    arr = df[ col ].sort( axis = 0, ascending = True, inplace = False ).unique()
    for a in arr :
        if a != None :
            list_spec.append( a )
    
sz = pd.Series( list_spec )    # series 처리
sz = sz.sort( axis = 0, ascending = True, inplace = False ).unique()    # sorting/ unique
sz = pd.Series( sz ).dropna()    # nan 처리
df_spec_meta = pd.DataFrame( sz, columns = [ 'spec' ] )
df_spec_meta.to_csv( './dataset/01.original.dataset/spec_meta.csv', index = False )

del( df )

.

#7. tube_end_form merge 시도

    * end_a, end_x는 tube_end_form dataset과 merge하여 forming feature를 확장할 수 있다
    .

In [26]:
df_tube_bill_specs[ 'end_a' ].value_counts()[:5]    # end_a의 % 확인

EF-003    11388
EF-018     2514
EF-008     2130
EF-017     1052
NONE        998
dtype: int64

* 압도적으로 EF-003, 018, 008, 017이 많다

In [27]:
df_tube_bill_specs[ 'end_x' ].value_counts()[:5]    # end_x의 % 확인

EF-003    10865
EF-018     2287
EF-017     2023
EF-008     1947
NONE       1377
dtype: int64

* 압도적으로 EF-003, 018, 017, 008이 많다

* <font color = 'red'><b>end_a, end_x 모두 NONE도 많다</b></font>

In [28]:
df_tube_bill_specs[ 'end_a' ].unique(), len( df_tube_bill_specs[ 'end_a' ].unique() )    # end_a의 unique

(array(['EF-003', 'EF-008', 'EF-021', 'NONE', 'EF-018', 'EF-005', 'EF-009',
        'EF-023', 'EF-017', 'EF-015', 'EF-012', 'EF-001', 'EF-010',
        'EF-004', 'EF-002', 'EF-013', 'EF-019', 'EF-007', 'EF-020',
        'EF-025', 'EF-016', 'EF-022', 'EF-011', 'EF-006', 'EF-014'], dtype=object),
 25)

In [29]:
df_tube_bill_specs[ 'end_x' ].unique(), len( df_tube_bill_specs[ 'end_x' ].unique() )    # end_x의 unique

(array(['EF-003', 'EF-008', 'EF-021', 'NONE', 'EF-018', 'EF-009', 'EF-017',
        'EF-019', 'EF-015', 'EF-012', 'EF-023', 'EF-002', 'EF-013',
        'EF-007', 'EF-010', 'EF-006', 'EF-016', 'EF-022', 'EF-004',
        'EF-001', 'EF-005', 'EF-011', 'EF-025', 'EF-014', 'EF-026', 'EF-024'], dtype=object),
 26)

In [30]:
df_tube_end_form = pd.read_csv( './dataset/01.original.dataset/tube_end_form.csv' )

In [31]:
df_tube_end_form.head( 3 )

,end_form_id,forming
0,EF-001,Yes
1,EF-002,No
2,EF-003,No


In [32]:
df_tube_end_form.tail( 3 )

,end_form_id,forming
25,EF-026,No
26,9999,No
27,NONE,No


* 9999가 존재한다 → df_tube_bill_spec내 'end_a', 'end_x' feature들은 NONE 정보가 있었으므로 이를 연계가능

In [33]:
df_tube_bill_specs[ [ 'tube_assembly_id', 'end_a', 'end_x' ] ].head( 3 )

,tube_assembly_id,end_a,end_x
0,TA-00001,EF-003,EF-003
1,TA-00002,EF-008,EF-008
2,TA-00003,EF-008,EF-008


In [34]:
# NONE값을 join위해, 표준으로 변환처리 → NONE은 없다는 의미로 받아들이고 분석을 진행
# df_tube_bill_specs[ 'end_a' ].replace( to_replace = 'NONE', value = np.nan, inplace = True )
# df_tube_bill_specs[ 'end_x' ].replace( to_replace = 'NONE', value = np.nan, inplace = True )

In [35]:
# df_tube_bill_specs[ [ 'tube_assembly_id', 'end_a', 'end_x' ] ]

In [36]:
# df_tube_bill_specs[ [ 'tube_assembly_id', 'end_a', 'end_x' ] ].head( 20 )

In [37]:
len( df_tube_bill_specs )

21198

In [38]:
# 동일 코드를 사용중인 end_a, end_x에 대해 연쇄적으로 merge를 수행
df_tube_bill_specs_end = df_tube_bill_specs.merge( df_tube_end_form, how = 'left',
                                                   left_on = 'end_a', right_on = 'end_form_id' )

In [39]:
len( df_tube_bill_specs_end )

21198

In [40]:
df_tube_bill_specs_end = df_tube_bill_specs_end.merge( df_tube_end_form, how = 'left',
                                                       left_on = 'end_x', right_on = 'end_form_id' )

In [41]:
len( df_tube_bill_specs_end )

21198

In [42]:
df_tube_bill_specs_end.sort( 'tube_assembly_id' ).head( 3 )

,tube_assembly_id,material_id,diameter,wall,length,num_bends,bend_radius,end_a_1x,end_a_2x,end_x_1x,...,SP-0091,SP-0092,SP-0093,SP-0094,SP-0095,SP-0096,end_form_id_x,forming_x,end_form_id_y,forming_y
0,TA-00001,SP-0035,12.70,1.65,164,5,38.10,0,0,0,...,0,0,0,0,0,0,EF-003,No,EF-003,No
1,TA-00002,SP-0019,6.35,0.71,137,8,19.05,0,0,0,...,0,0,0,0,0,0,EF-008,Yes,EF-008,Yes
2,TA-00003,SP-0019,6.35,0.71,127,7,19.05,0,0,0,...,0,0,0,0,0,0,EF-008,Yes,EF-008,Yes


In [43]:
df_tube_bill_specs_end.drop( [ 'end_form_id_x', 'end_form_id_y' ], axis = 1, inplace = True )

In [44]:
df_tube_bill_specs_end.head( 3 )

,tube_assembly_id,material_id,diameter,wall,length,num_bends,bend_radius,end_a_1x,end_a_2x,end_x_1x,...,SP-0089,SP-0090,SP-0091,SP-0092,SP-0093,SP-0094,SP-0095,SP-0096,forming_x,forming_y
0,TA-00001,SP-0035,12.70,1.65,164,5,38.10,0,0,0,...,0,0,0,0,0,0,0,0,No,No
1,TA-00002,SP-0019,6.35,0.71,137,8,19.05,0,0,0,...,0,0,0,0,0,0,0,0,Yes,Yes
2,TA-00003,SP-0019,6.35,0.71,127,7,19.05,0,0,0,...,0,0,0,0,0,0,0,0,Yes,Yes


In [45]:
dic = { 'Yes' : 'Y', 'No' : 'N' }
df_tube_bill_specs_end.replace( dic, inplace = True )

In [46]:
df_tube_bill_specs_end.head( 3 )

,tube_assembly_id,material_id,diameter,wall,length,num_bends,bend_radius,end_a_1x,end_a_2x,end_x_1x,...,SP-0089,SP-0090,SP-0091,SP-0092,SP-0093,SP-0094,SP-0095,SP-0096,forming_x,forming_y
0,TA-00001,SP-0035,12.70,1.65,164,5,38.10,0,0,0,...,0,0,0,0,0,0,0,0,N,N
1,TA-00002,SP-0019,6.35,0.71,137,8,19.05,0,0,0,...,0,0,0,0,0,0,0,0,Y,Y
2,TA-00003,SP-0019,6.35,0.71,127,7,19.05,0,0,0,...,0,0,0,0,0,0,0,0,Y,Y


In [47]:
df_tube_bill_specs_end.sort( 'tube_assembly_id' ).to_csv( './dataset/03.merged/tube_bill_specs_end_merged.csv', index = False )

In [48]:
df_tube_bill_specs_end.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21198 entries, 0 to 21197
Columns: 180 entries, tube_assembly_id to forming_y
dtypes: float64(50), int64(106), object(24)
memory usage: 29.3+ MB


In [49]:
df_tube_bill_specs_end.count()

tube_assembly_id    21198
material_id         21198
diameter            21198
wall                21198
length              21198
num_bends           21198
bend_radius         21198
end_a_1x            21198
end_a_2x            21198
end_x_1x            21198
end_x_2x            21198
end_a               21198
end_x               21198
num_boss            21198
num_bracket         21198
...
SP-0084      21198
SP-0085      21198
SP-0086      21198
SP-0087      21198
SP-0088      21198
SP-0089      21198
SP-0090      21198
SP-0091      21198
SP-0092      21198
SP-0093      21198
SP-0094      21198
SP-0095      21198
SP-0096      21198
forming_x    21198
forming_y    21198
Length: 180, dtype: int64

In [50]:
del( df_tube_bill_specs )
del( df_tube_end_form )
del( df_tube_bill_specs_end )